In [1]:
import requests
import pandas as pd
from datetime import datetime
import time
from typing import List, Dict
import logging

class PBPStatsAPI:
    def __init__(self, start_year: int = 2013, end_year: int = 2024):
        self.base_url = "https://api.pbpstats.com/get-game-logs/nba?"
        self.season_types = ["Regular Season", "Playoffs"]
        self.start_year = start_year
        self.end_year = end_year

    def get_season_years(self) -> List[str]:
        """Generate season strings from start_year to end_year."""
        return [f"{year}-{str(year + 1)[2:]}" for year in range(self.start_year, self.end_year + 1)]
    
    def get_team_game_logs(self, team_id: str) -> pd.DataFrame:
        """
        Fetch all game logs for a specific team within the specified year range.
        
        Args:
            team_id (str): The team ID in PBP Stats format
            
        Returns:
            pd.DataFrame: DataFrame containing all game logs
        """
        all_games = []
        seasons = self.get_season_years()
        
        for season in seasons:
            for season_type in self.season_types:
                if season_type != 'Playoffs':
                    try:
                        params = {
                            "Season": season,
                            "SeasonType": season_type,
                            "EntityId": team_id,
                            "EntityType": "Team"
                        }
                        
                        response = requests.get(self.base_url, params=params)
                        data_response = response.json()
                        raw_data = data_response["multi_row_table_data"]
                        games_data = pd.DataFrame(raw_data)
                        
                        games_data['Season'] = season
                        games_data['year'] = int(season.split('-')[0]) + 1
                        games_data['SeasonType'] = season_type
                        all_games.append(games_data)
                                
                        # Respect API rate limits
                        time.sleep(1.2)
                        
                    except requests.exceptions.RequestException as e:
                        logging.error(f"Error fetching data for {team_id} in {season} {season_type}: {str(e)}")
                        continue
                    
        return pd.concat(all_games) if all_games else pd.DataFrame()

def fetch_all_teams_game_logs(team_ids: List[str], start_year: int, end_year: int) -> Dict[str, pd.DataFrame]:
    """
    Fetch game logs for multiple teams within the specified year range.
    
    Args:
        team_ids (List[str]): List of team IDs
        start_year (int): Starting year for data collection
        end_year (int): Ending year for data collection
        
    Returns:
        Dict[str, pd.DataFrame]: Dictionary with team IDs as keys and their game logs as values
    """
    api = PBPStatsAPI(start_year=start_year, end_year=end_year)
    team_games = {}
    
    for team_id in team_ids:
        logging.info(f"Fetching game logs for team {team_id}")
        team_games[team_id] = api.get_team_game_logs(team_id)
        
    return team_games

# Example usage
if __name__ == "__main__":
    # Example team IDs (you'll need to use the correct IDs from PBP Stats)
    df = pd.read_csv('index_master.csv')
    df = df[df.year > 2013]
    df = df[df.team != 'TOT']
    team_ids = df['team_id'].unique().tolist()

    # Input start and end years
    start_year = 2013
    end_year =2014

    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    
    # Fetch game logs
    team_game_logs = fetch_all_teams_game_logs(team_ids, start_year, end_year)
    
    # Example: Save to CSV files
    for team_id, games_df in team_game_logs.items():
        if not games_df.empty:
            for year in range(start_year, end_year + 1):
                team_df = games_df[games_df.year == year]
                if not team_df.empty:
                    team_df.to_csv(f"team/{year}/{team_id}.csv", index=False)
            logging.info(f"Saved game logs for {team_id}")


2024-12-12 09:07:50,587 - INFO - Fetching game logs for team 1610612761
2024-12-12 09:07:57,867 - INFO - Fetching game logs for team 1610612758
2024-12-12 09:08:06,389 - INFO - Fetching game logs for team 1610612760
2024-12-12 09:08:14,073 - INFO - Fetching game logs for team 1610612766
2024-12-12 09:08:21,249 - INFO - Fetching game logs for team 1610612749
2024-12-12 09:08:29,172 - INFO - Fetching game logs for team 1610612753
2024-12-12 09:08:36,623 - INFO - Fetching game logs for team 1610612740
2024-12-12 09:08:43,692 - INFO - Fetching game logs for team 1610612752
2024-12-12 09:08:51,329 - INFO - Fetching game logs for team 1610612757
2024-12-12 09:08:58,899 - INFO - Fetching game logs for team 1610612755
2024-12-12 09:09:06,394 - INFO - Fetching game logs for team 1610612754
2024-12-12 09:09:14,065 - INFO - Fetching game logs for team 1610612748
2024-12-12 09:09:21,171 - INFO - Fetching game logs for team 1610612763
2024-12-12 09:09:28,156 - INFO - Fetching game logs for team 161

In [2]:
for team_id, games_df in team_game_logs.items():
    
    if not games_df.empty:
        for year in range(2014,2026):
            team_df=games_df[games_df.year==year]
            team_df.to_csv(f"team/{year}/{team_id}.csv", index=False)
        logging.info(f"Saved game logs for {team_id}")

2024-12-12 09:11:37,167 - INFO - Saved game logs for 1610612761
2024-12-12 09:11:37,227 - INFO - Saved game logs for 1610612758
2024-12-12 09:11:37,264 - INFO - Saved game logs for 1610612760
2024-12-12 09:11:37,308 - INFO - Saved game logs for 1610612766
2024-12-12 09:11:37,351 - INFO - Saved game logs for 1610612749
2024-12-12 09:11:37,387 - INFO - Saved game logs for 1610612753
2024-12-12 09:11:37,424 - INFO - Saved game logs for 1610612740
2024-12-12 09:11:37,467 - INFO - Saved game logs for 1610612752
2024-12-12 09:11:37,498 - INFO - Saved game logs for 1610612757
2024-12-12 09:11:37,533 - INFO - Saved game logs for 1610612755
2024-12-12 09:11:37,564 - INFO - Saved game logs for 1610612754
2024-12-12 09:11:37,596 - INFO - Saved game logs for 1610612748
2024-12-12 09:11:37,632 - INFO - Saved game logs for 1610612763
2024-12-12 09:11:37,663 - INFO - Saved game logs for 1610612741
2024-12-12 09:11:37,692 - INFO - Saved game logs for 1610612751
2024-12-12 09:11:37,722 - INFO - Saved g